<a href="https://colab.research.google.com/github/qkrwoghd04/multimodal_learning/blob/transformer-practice/Attention_is_all_you_need%20practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Attention is all you need(NIPS 2017) practice**

Git: https://github.com/qkrwoghd04

In [52]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
%matplotlib inline

In [54]:
#torchtext 0.9.0 버전 이후에 torchtext.data → torchtext.legacy.data 로 변경됨
#torchtext 0.12.0 버전 이후 legacy package가 제거되었음
!pip install torchtext==0.6.0

In [55]:
!python -m spacy download de_core_news_sm
#OSError: [E050] Can't find model 'de_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 31.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


**데이터 전처리(Preprocessing)**
<br>영어와 독일어 전처리 모듈 설치



In [56]:
!pip install spacy

In [57]:
import spacy

spacy_en = spacy.load('en_core_web_sm')
spacy_de = spacy.load('de_core_news_sm')

In [58]:
import en_core_web_sm, de_core_news_sm

In [59]:
nlp_en = en_core_web_sm.load()
nlp_de = de_core_news_sm.load()

In [60]:
tokenized = nlp_en('I am a student.') #토큰화 기능 사용

In [61]:
for i, token in enumerate(tokenized):
  print(f'인덱스 {i}: {token.text}')

인덱스 0: I
인덱스 1: am
인덱스 2: a
인덱스 3: student
인덱스 4: .


In [62]:
def tokenize_en(text):
  return [token.text for token in spacy_en.tokenizer(text)]

def tokenize_de(text):
  return [token.text for token in spacy_de.tokenizer(text)]

필드(field) lib을 통해 데이터셋에 대한 specific한 전처리 내용 명시

In [63]:
!ls -al /content/drive

total 16
dr-x------  2 root root 4096 Mar 13 03:29 .file-revisions-by-id
drwx------ 13 root root 4096 Mar 13 03:29 MyDrive
dr-x------  2 root root 4096 Mar 13 03:29 .shortcut-targets-by-id
drwx------  5 root root 4096 Mar 13 03:29 .Trash-0


In [64]:
from torchtext.data import Field, Example, Dataset
from torchtext.data import BucketIterator

SRC = Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<eos>', lower=True)
TRG = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>', lower=True)

In [65]:
def build_dataset(path, src_ext, trg_ext, src_field, trg_field):
    src_path = f"{path}/{src_ext}"
    trg_path = f"{path}/{trg_ext}"

    examples = []
    with open(src_path, encoding='utf-8') as src_file, open(trg_path, encoding='utf-8') as trg_file:
        for src_line, trg_line in zip(src_file, trg_file):
            src_line, trg_line = src_line.strip(), trg_line.strip()
            if src_line != '' and trg_line != '':
                examples.append(Example.fromlist([src_line, trg_line], fields=[('src', src_field), ('trg', trg_field)]))

    return Dataset(examples, fields=[('src', src_field), ('trg', trg_field)])


In [66]:
!ls "/content/drive/MyDrive/Datasets/multi30k-dataset/data/task1/raw"

test_2016_flickr.cs.gz	test_2017_flickr.fr.gz	test_2018_flickr.de.gz	train.de.gz  val.de.gz
test_2016_flickr.de.gz	test_2017_mscoco.de.gz	test_2018_flickr.en	train.en     val.en
test_2016_flickr.en.gz	test_2017_mscoco.en.gz	test_2018_flickr.en.gz	train.en.gz  val.en.gz
test_2016_flickr.fr.gz	test_2017_mscoco.fr.gz	test_2018_flickr.fr.gz	train.fr.gz  val.fr.gz
test_2017_flickr.de.gz	test_2018_flickr.cs.gz	train.cs.gz		val.cs.gz
test_2017_flickr.en.gz	test_2018_flickr.de	train.de		val.de


In [67]:
# 데이터셋 생성
train_data = build_dataset('/content/drive/MyDrive/Datasets/multi30k-dataset/data/task1/raw', 'train.de', 'train.en', SRC, TRG)
valid_data = build_dataset('/content/drive/MyDrive/Datasets/multi30k-dataset/data/task1/raw', 'val.de', 'val.en', SRC, TRG)
test_data = build_dataset('/content/drive/MyDrive/Datasets/multi30k-dataset/data/task1/raw', 'test_2018_flickr.de', 'test_2018_flickr.en', SRC, TRG)
# 테스트 데이터셋도 같은 방식으로 생성할 수 있습니다. 파일 경로만 변경하세요.


In [68]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of test examples: {len(test_data.examples)}")
# 테스트 데이터셋의 경우도 동일하게 적용 가능, 예시: print(f"Number of testing examples: {len(test_data.examples)}")


Number of training examples: 28808
Number of validation examples: 1014
Number of test examples: 1071


In [69]:
print(vars(train_data.examples[30])['src'])
print(vars(train_data.examples[30])['trg'])

['ein', 'mann', ',', 'der', 'mit', 'einer', 'tasse', 'kaffee', 'an', 'einem', 'urinal', 'steht', '.']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


In [70]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

In [71]:
print(f'len(SRC): {len(SRC.vocab)}')
print(f'len(SRC): {len(TRG.vocab)}')

len(SRC): 7822
len(SRC): 5882


In [72]:
print(TRG.vocab.stoi['abcabc'])
print(TRG.vocab.stoi[TRG.pad_token])
print(TRG.vocab.stoi['<sos>'])
print(TRG.vocab.stoi['<eos>'])
print(TRG.vocab.stoi['hello'])
print(TRG.vocab.stoi['world'])

0
1
2
3
4099
1749


In [73]:
!pip install torch

In [97]:
import torch
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 일반적인 데이터 로더(data loader)의 iterator와 유사하게 사용가능
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device,
    sort=False,  # 데이터를 정렬하지 않도록 설정
    sort_within_batch=False,  # 배치 내에서도 데이터를 정렬하지 않도록 설정
    shuffle=True  # 데이터 셔플링은 유지 (필요에 따라 조정)
)

In [98]:
# 첫 번째 배치에서 첫 번째 문장 확인
first_batch = next(iter(train_iterator))
src, trg = first_batch.src, first_batch.trg

# SRC 필드의 vocab을 사용하여 인덱스를 실제 텍스트로 변환
src_vocab = SRC.vocab.itos  # index to string

# 첫 번째 문장의 첫 번째 단어부터 순회
first_sentence = [src_vocab[index] for index in src[:, 0].tolist()]  # 첫 번째 문장에서, 모든 단어에 대해
print('첫 번째 문장:', ' '.join(first_sentence))


첫 번째 문장: <sos> zwei frauen auf einer couch , die frau in weiß scheint etwas zu erklären , das sie in der hand hält . <eos> <pad> <pad> <pad> <pad> <pad> <pad>


#### 만약 torch.Size([30, 128])와 같이 배치 차원이 반대로 출력이 된다면, **BucketIterator 구현에서 배치 차원이 첫 번째에 오도록 설정되어 있을 가능성이 높다** permute 함수를 통해 차원을 변경할 수 있다

In [99]:
for i,batch in enumerate(train_iterator):
  src = batch.src
  trg = batch.trg

  # 차원 변경: [sequence length, batch size] -> [batch size, sequence length]
  src = src.permute(1, 0)
  trg = trg.permute(1, 0)

  print(f"첫 번째 배치 크기: {src.shape}")

  for i in range(src.shape[1]):
    print(f"인덱스 {i}: {src[0][i].item()}")

  break


첫 번째 배치 크기: torch.Size([128, 27])
인덱스 0: 2
인덱스 1: 8
인덱스 2: 167
인덱스 3: 36
인덱스 4: 22
인덱스 5: 278
인덱스 6: 545
인덱스 7: 8
인덱스 8: 34
인덱스 9: 7
인덱스 10: 15
인덱스 11: 152
인덱스 12: 4
인덱스 13: 3
인덱스 14: 1
인덱스 15: 1
인덱스 16: 1
인덱스 17: 1
인덱스 18: 1
인덱스 19: 1
인덱스 20: 1
인덱스 21: 1
인덱스 22: 1
인덱스 23: 1
인덱스 24: 1
인덱스 25: 1
인덱스 26: 1


### Multi Head Attention Architecture

*   Attention takes input for 3 factors

  *   queires
  *   keys
  *   values


*   하이퍼 파라미터

  *   hidden_dim: 하나의 단어에 대한 임베딩 차원
  *   n_heads: 헤드의 개수 = saled dot-product attention의 개수
  *   dropout_ratio: 드롭아웃 비율












In [100]:
import torch.nn as nn

class MultiHeadAttentionLayer(nn.Module):
  def __init__(self, hidden_dim, n_heads, dropout_ratio, device):
    super().__init__()

    assert hidden_dim % n_heads == 0

    self.hidden_dim = hidden_dim
    self.n_heads = n_heads
    self.head_dim = hidden_dim// n_heads

    self.fc_q = nn.Linear(hidden_dim, hidden_dim)
    self.fc_k = nn.Linear(hidden_dim, hidden_dim)
    self.fc_v = nn.Linear(hidden_dim, hidden_dim)

    self.fc_o = nn.Linear(hidden_dim, hidden_dim)

    self.dropout = nn.Dropout(dropout_ratio)

    self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

  def forward(self, query, key, value, mask = None):

    batch_size = query.shape[0]

    Q = self.fc_q(query)
    K = self.fc_k(key)
    V = self.fc_v(value)

    Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)
    K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)
    V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)

    # Attention Energy 계산
    energy = torch.matmul(Q, K.permute(0,1,3,2))/ self.scale

    if mask is not None:
      energy = energy.masked_fill(mask==0, -1e10)

    attention = torch.softmax(energy, dim=-1)

    x = torch.matmul(self.dropout(attention), V)

    x = x.permute(0,2,1,3).contiguous()

    x = x.view(batch_size, -1, self.hidden_dim)

    x = self.fc_o(x)

    return x, attention

In [101]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hidden_dim, pf_dim, dropout_ratio):
        super().__init__()

        self.fc_1 = nn.Linear(hidden_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hidden_dim)

        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, x):

        # x: [batch_size, seq_len, hidden_dim]

        x = self.dropout(torch.relu(self.fc_1(x)))

        # x: [batch_size, seq_len, pf_dim]

        x = self.fc_2(x)

        # x: [batch_size, seq_len, hidden_dim]

        return x

In [102]:
class EncoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

    # 하나의 임베딩이 복제되어 Query, Key, Value로 입력되는 방식
    def forward(self, src, src_mask):

        # src: [batch_size, src_len, hidden_dim]
        # src_mask: [batch_size, src_len]

        # self attention
        # 필요한 경우 마스크(mask) 행렬을 이용하여 어텐션(attention)할 단어를 조절 가능
        _src, _ = self.self_attention(src, src, src, src_mask)

        # dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))

        # src: [batch_size, src_len, hidden_dim]

        # position-wise feedforward
        _src = self.positionwise_feedforward(src)

        # dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))

        # src: [batch_size, src_len, hidden_dim]

        return src

In [103]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=100):
        super().__init__()

        self.device = device

        self.tok_embedding = nn.Embedding(input_dim, hidden_dim)
        self.pos_embedding = nn.Embedding(max_length, hidden_dim)

        self.layers = nn.ModuleList([EncoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    def forward(self, src, src_mask):

        # src: [batch_size, src_len]
        # src_mask: [batch_size, src_len]

        batch_size = src.shape[0]
        src_len = src.shape[1]

        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

        # pos: [batch_size, src_len]

        # 소스 문장의 임베딩과 위치 임베딩을 더한 것을 사용
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))

        # src: [batch_size, src_len, hidden_dim]

        # 모든 인코더 레이어를 차례대로 거치면서 순전파(forward) 수행
        for layer in self.layers:
            src = layer(src, src_mask)

        # src: [batch_size, src_len, hidden_dim]

        return src # 마지막 레이어의 출력을 반환

In [104]:
class DecoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.encoder_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더의 출력 값(enc_src)을 어텐션(attention)하는 구조
    def forward(self, trg, enc_src, trg_mask, src_mask):

        # trg: [batch_size, trg_len, hidden_dim]
        # enc_src: [batch_size, src_len, hidden_dim]
        # trg_mask: [batch_size, trg_len]
        # src_mask: [batch_size, src_len]

        # self attention
        # 자기 자신에 대하여 어텐션(attention)
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)

        # dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))

        # trg: [batch_size, trg_len, hidden_dim]

        # encoder attention
        # 디코더의 쿼리(Query)를 이용해 인코더를 어텐션(attention)
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)

        # dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))

        # trg: [batch_size, trg_len, hidden_dim]

        # positionwise feedforward
        _trg = self.positionwise_feedforward(trg)

        # dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))

        # trg: [batch_size, trg_len, hidden_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        return trg, attention

In [105]:
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=100):
        super().__init__()

        self.device = device

        self.tok_embedding = nn.Embedding(output_dim, hidden_dim)
        self.pos_embedding = nn.Embedding(max_length, hidden_dim)

        self.layers = nn.ModuleList([DecoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])

        self.fc_out = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    def forward(self, trg, enc_src, trg_mask, src_mask):

        # trg: [batch_size, trg_len]
        # enc_src: [batch_size, src_len, hidden_dim]
        # trg_mask: [batch_size, trg_len]
        # src_mask: [batch_size, src_len]

        batch_size = trg.shape[0]
        trg_len = trg.shape[1]

        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

        # pos: [batch_size, trg_len]

        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))

        # trg: [batch_size, trg_len, hidden_dim]

        for layer in self.layers:
            # 소스 마스크와 타겟 마스크 모두 사용
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)

        # trg: [batch_size, trg_len, hidden_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        output = self.fc_out(trg)

        # output: [batch_size, trg_len, output_dim]

        return output, attention


In [106]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    # 소스 문장의  토큰에 대하여 마스크(mask) 값을 0으로 설정
    def make_src_mask(self, src):

        # src: [batch_size, src_len]

        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        # src_mask: [batch_size, 1, 1, src_len]

        return src_mask

    # 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용
    def make_trg_mask(self, trg):

        # trg: [batch_size, trg_len]

        """ (마스크 예시)
        1 0 0 0 0
        1 1 0 0 0
        1 1 1 0 0
        1 1 1 0 0
        1 1 1 0 0
        """
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)

        # trg_pad_mask: [batch_size, 1, 1, trg_len]

        trg_len = trg.shape[1]

        """ (마스크 예시)
        1 0 0 0 0
        1 1 0 0 0
        1 1 1 0 0
        1 1 1 1 0
        1 1 1 1 1
        """
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()

        # trg_sub_mask: [trg_len, trg_len]

        trg_mask = trg_pad_mask & trg_sub_mask

        # trg_mask: [batch_size, 1, trg_len, trg_len]

        return trg_mask

    def forward(self, src, trg):

        # src: [batch_size, src_len]
        # trg: [batch_size, trg_len]

        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)

        # src_mask: [batch_size, 1, 1, src_len]
        # trg_mask: [batch_size, 1, trg_len, trg_len]

        enc_src = self.encoder(src, src_mask)

        # enc_src: [batch_size, src_len, hidden_dim]

        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)

        # output: [batch_size, trg_len, output_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        return output, attention

In [107]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HIDDEN_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

In [108]:
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, HIDDEN_DIM, ENC_LAYERS, ENC_HEADS, ENC_PF_DIM, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, HIDDEN_DIM, DEC_LAYERS, DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, device)

# Transformer 객체 선언
model = Transformer(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)


In [109]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,024,762 trainable parameters


In [110]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

Transformer(
  (encoder): Encoder(
    (tok_embedding): Embedding(7822, 256)
    (pos_embedding): Embedding(100, 256)
    (layers): ModuleList(
      (0-2): 3 x EncoderLayer(
        (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ff_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attention): MultiHeadAttentionLayer(
          (fc_q): Linear(in_features=256, out_features=256, bias=True)
          (fc_k): Linear(in_features=256, out_features=256, bias=True)
          (fc_v): Linear(in_features=256, out_features=256, bias=True)
          (fc_o): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (positionwise_feedforward): PositionwiseFeedforwardLayer(
          (fc_1): Linear(in_features=256, out_features=512, bias=True)
          (fc_2): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
     

In [111]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
LEARNING_RATE = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [123]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()  # 학습 모드
    epoch_loss = 0

    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg

        # 중간 상태 출력: 소스 및 타겟의 배치 사이즈와 최대 인덱스 값 확인
        print(f"Batch {i+1}:")
        print(f"  src size: {src.size()}, trg size: {trg.size()}")
        print(f"  max src index: {src.max().item()}, max trg index: {trg.max().item()}")

        optimizer.zero_grad()

        # 모델의 출력 결과
        output, _ = model(src, trg[:, :-1])

        # 출력 차원
        output_dim = output.shape[-1]

        # 출력 및 타겟의 재구성
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:, 1:].contiguous().view(-1)

        # 손실 계산
        loss = criterion(output, trg)
        loss.backward()

        # 기울기 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # 파라미터 업데이트
        optimizer.step()

        epoch_loss += loss.item()

        # 배치 처리 후 상태 출력
        print(f"  Loss: {loss.item()}")

        # 오류를 분석하기 위해 첫 배치만 실행하고 중단
        if i == 0:
            break

    return epoch_loss / len(iterator)


In [124]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0

    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            # 출력 단어의 마지막 인덱스()는 제외
            # 입력을 할 때는 부터 시작하도록 처리
            output, _ = model(src, trg[:,:-1])

            # output: [배치 크기, trg_len - 1, output_dim]
            # trg: [배치 크기, trg_len]

            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)
            # 출력 단어의 인덱스 0()은 제외
            trg = trg[:,1:].contiguous().view(-1)

            # output: [배치 크기 * trg_len - 1, output_dim]
            # trg: [배치 크기 * trg len - 1]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


In [125]:
import math
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [126]:
import time
import math
import random

N_EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'transformer_german_to_english.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

Batch 1:
  src size: torch.Size([27, 128]), trg size: torch.Size([28, 128])
  max src index: 7793, max trg index: 5806


IndexError: index out of range in self

In [95]:
# 어휘 사전 크기 확인
print(f"입력 어휘 사전 크기(SRC): {len(SRC.vocab)}")
print(f"출력 어휘 사전 크기(TRG): {len(TRG.vocab)}")

# 임베딩 행렬 크기 확인
print(f"인코더 토큰 임베딩 행렬 크기: {model.encoder.tok_embedding.weight.shape}")
print(f"디코더 토큰 임베딩 행렬 크기: {model.decoder.tok_embedding.weight.shape}")

# 입력 데이터의 최대 인덱스 확인
# 훈련 데이터에서 소스 및 타겟 어휘의 최대 인덱스 값을 검사합니다.
max_src_index = max([max([SRC.vocab.stoi[token] for token in example.src]) for example in train_data.examples])
max_trg_index = max([max([TRG.vocab.stoi[token] for token in example.trg]) for example in train_data.examples])

print(f"훈련 데이터 소스 어휘의 최대 인덱스: {max_src_index}")
print(f"훈련 데이터 타겟 어휘의 최대 인덱스: {max_trg_index}")


입력 어휘 사전 크기(SRC): 7822
출력 어휘 사전 크기(TRG): 5882
인코더 토큰 임베딩 행렬 크기: torch.Size([7822, 256])
디코더 토큰 임베딩 행렬 크기: torch.Size([5882, 256])
훈련 데이터 소스 어휘의 최대 인덱스: 7821
훈련 데이터 타겟 어휘의 최대 인덱스: 5881
